In [2]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

In [3]:
margin = 3
sample_size = 60000
batchSize = 40

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [5]:
toTensor = transforms.ToTensor()


xSub = x_train[:sample_size]
ySub = y_train[:sample_size]

xTensor = toTensor(xSub)
xTensor = xTensor.permute(1, 0, 2).reshape(len(xSub), 1, 28, 28) # shape: (28, sample_size, 28) -->  shape: (sample_size, 1, 28, 28)

In [6]:
model = nn.Sequential(  nn.Flatten(),
                        nn.Linear(28*28,256),
                        nn.ReLU(),
                        nn.Linear(256,32),
                        nn.Sigmoid()
                        )

#optimizer = optim.SGD(model.parameters())


In [7]:
results = model(xTensor)

In [8]:
results = (results > 0.5).to(int)

In [9]:
results

tensor([[0, 1, 1,  ..., 1, 1, 1],
        [0, 1, 0,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 1, 1, 1],
        ...,
        [0, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 1, 1, 1]])

In [10]:
n = 60000
S = [[0] * n for _ in range(n)]

for i in range(n):
    for j in range(n):
        hamming_distance = torch.sum(results[i] != results[j])
        if hamming_distance < 9:
            S[i][j] = 1  # Set S[i][j] to 1 if hamming distance < 9
        else:
            S[i][j] = 0  # Set S[i][j] to 0 if hamming distance >= 9

S

MemoryError: 

## CUSTOM LOSS IMPLEMENTATION

In [ ]:
s_ij = 1
u_i = 1
u_j = 1
b_i = 1
v = 1
W = 1
eta = 1

theta_ij = 1/2 * u_i.transpose() * u_j

In [ ]:
s_ij = 

In [7]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs):
        loss = - sum(s_ij * theta_ij - math.log(1 + math.e^theta_ij)) + eta * sum(norm(b_i - (W.transpose() * inputs + v))^2)    
        return loss.mean()